# 라이브러리 호출

In [58]:
import os
import re
import shutil
import zipfile
import urllib3
import unicodedata
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Embedding, GRU, Dense, Input, LSTM, Embedding, Dense, Masking

In [59]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 데이터

## 샘플 개수

In [60]:
num_samples = 30000

## 데이터 전처리

In [61]:
def to_ascii(s):
    # 프랑스어 악센트(accent) 삭제
    # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess_sentence(sent):
    # 악센트 제거 함수 호출
    sent = to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백 추가.
    # ex) "I am a student." => "I am a student ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [62]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


## 인코더, 디코더 데이터 분리

In [63]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("D:/Dataset/dataset/parallel_corpus/fra-eng.txt", "r", encoding="utf-8") as lines:
        for i, line in enumerate(lines):
            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [64]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>']]


## 데이터 토큰화

In [65]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
encoder_input = pad_sequences(encoder_input, padding="post")

tokenizer_fra = Tokenizer(filters="", lower=False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)

decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_input = pad_sequences(decoder_input, padding="post")

decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
decoder_target = pad_sequences(decoder_target, padding="post")

In [66]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (30000, 7)
디코더의 입력의 크기(shape) : (30000, 16)
디코더의 레이블의 크기(shape) : (30000, 16)


In [67]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 4380, 프랑스어 단어 집합의 크기 : 7674


## 딕셔너리 생성

In [68]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [69]:
## 데이터 셔플
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

## 훈련 / 테스트 데이터 분리

In [70]:
n_of_val = int(30000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3000


In [71]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [72]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (27000, 7)
훈련 target 데이터의 크기 : (27000, 16)
훈련 target 레이블의 크기 : (27000, 16)
테스트 source 데이터의 크기 : (3000, 7)
테스트 target 데이터의 크기 : (3000, 16)
테스트 target 레이블의 크기 : (3000, 16)


In [73]:
print('훈련 source 데이터의 샘플 :',encoder_input_train[0])
print('훈련 target 데이터의 샘플 :',decoder_input_train[0])
print('훈련 target 레이블의 샘플 :',decoder_target_train[0])
print('테스트 source 데이터의 샘플 :',encoder_input_test[0])
print('테스트 target 데이터의 샘플 :',decoder_input_test[0])
print('테스트 target 레이블의 샘플 :',decoder_target_test[0])

훈련 source 데이터의 샘플 : [  6  19 854   1   0   0   0]
훈련 target 데이터의 샘플 : [  2  31   8  61 528   1   0   0   0   0   0   0   0   0   0   0]
훈련 target 레이블의 샘플 : [ 31   8  61 528   1   3   0   0   0   0   0   0   0   0   0   0]
테스트 source 데이터의 샘플 : [  2 358 350   3   1   0   0]
테스트 target 데이터의 샘플 : [  2   4   9  23  14 285 120   1   0   0   0   0   0   0   0   0]
테스트 target 레이블의 샘플 : [  4   9  23  14 285 120   1   3   0   0   0   0   0   0   0   0]


# 번역기 제작

In [74]:
embedding_dim = 64
hidden_units = 64

In [75]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, hidden_units) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력을 정의.
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [76]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train,
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=256, epochs=50)

Epoch 1/50
106/106 [==============================] - 13s 48ms/step - loss: 6.9711 - acc: 0.5668 - val_loss: 2.2720 - val_acc: 0.6214
Epoch 2/50
106/106 [==============================] - 3s 26ms/step - loss: 2.1822 - acc: 0.6206 - val_loss: 1.9846 - val_acc: 0.6345
Epoch 3/50
106/106 [==============================] - 3s 26ms/step - loss: 1.9310 - acc: 0.6444 - val_loss: 1.8077 - val_acc: 0.6785
Epoch 4/50
106/106 [==============================] - 3s 26ms/step - loss: 1.7736 - acc: 0.6993 - val_loss: 1.7112 - val_acc: 0.7415
Epoch 5/50
106/106 [==============================] - 3s 26ms/step - loss: 1.6786 - acc: 0.7458 - val_loss: 1.6212 - val_acc: 0.7484
Epoch 6/50
106/106 [==============================] - 3s 26ms/step - loss: 1.5829 - acc: 0.7498 - val_loss: 1.5364 - val_acc: 0.7604
Epoch 7/50
106/106 [==============================] - 3s 26ms/step - loss: 1.5094 - acc: 0.7597 - val_loss: 1.4720 - val_acc: 0.7630
Epoch 8/50
106/106 [==============================] - 3s 26ms/step -

# 번역기 작동

In [82]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계 시작
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [83]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] +
                                                    states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' ' + sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [84]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if (encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence


# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if (encoded_word != 0 and encoded_word != tar_to_index['<sos>']
                and encoded_word != tar_to_index['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
    return sentence

In [85]:
import pandas as pd

In [97]:
english = [] # 영어
french_ref = [] # 프랑스어 정답
french_cand = [] # 프랑스어 기계번역

In [98]:
for seq_index in range(1000):
    try:
        input_seq = encoder_input_test[seq_index:seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)

        english.append(seq_to_src(encoder_input_test[seq_index]))
        french_ref.append(seq_to_tar(decoder_input_test[seq_index]))
        french_cand.append(decoded_sentence[1:-5])
    except:
        pass

In [99]:
len(english)

1000

In [90]:
pd.DataFrame({"English":english, "French_cand":french_cand, "French_ref":french_ref}).to_csv("번역결과.csv", header=True, encoding="utf-8")

# 번역 결과 확인

In [91]:
import pandas as pd
import nltk.translate.bleu_score as bleu

In [92]:
result = pd.read_csv("번역결과.csv", index_col=0)

In [93]:
result

,English,French_cand,French_ref
0,i already told you .,je vous ai vu ca .,je vous l ai deja dit .
1,dinner s ready .,la avons la maison .,le diner est pret .
2,take care !,prends soin de vous !,prenez soin de vous .
3,what a team !,comme c est a t il !,quelle equipe !
4,did anyone miss me ?,qu est ce que tu a fait mal a l ecole ?,ai je manque a quiconque ?
...,...,...,...
995,he fills the bill .,il a l air triste .,il paie l addition .
996,tom lies .,tom .,tom ment .
997,you re so mean .,vous etes si tu heureux .,vous etes si mechantes .
998,i m not panicking .,je ne me suis pas naif .,je ne panique pas .


In [94]:
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands'
]

print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

패키지 NLTK의 BLEU : 0.41180376356915777


In [95]:
score_list = []
for idx, data in result.iterrows():
    temp = []
    temp2 = []
    a = data["French_ref"].split()
    temp.append(a)
    b = data["French_cand"].split()
    temp2.append(b)
    score = bleu.sentence_bleu(temp,b)
    score_list.append(score)

C:\Users\user\miniconda3\envs\deep_tf\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\user\miniconda3\envs\deep_tf\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\user\miniconda3\envs\deep_tf\lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram or

In [96]:
sum(score_list)/len(score_list)

0.03824374930430969